# Manual Trigger of Human review process for any Document
This notebook shows you how to manually trigger human review process of any document (when the Validation filter is set to no-filter in the human review configuration).

In [ ]:
# Install necessary Python libraries and restart your kernel after.
!pip install -r ../requirements.txt

In [ ]:
from google.cloud import documentai_v1beta3 as documentai
from PIL import Image, ImageDraw

import os
import pandas as pd

## Set your Processor Variables 

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID_HERE"
LOCATION = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = "PROCESSOR_ID"  # Create processor in Cloud Console
DOCUMENT_PATH = "../resources/general/form.tiff" # Update to path of target document

The following code calls the synchronous API and parses the form fields and values.

In [ ]:
def review_document_sample():
    # Instantiates a client
    
    client_options = {"api_endpoint": "{}-documentai.googleapis.com".format(LOCATION)}
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id/humanReviewConfig
    # You must create new processors in the Cloud Console first
    name = f"projects/{PROJECT_ID}/locations/{LOCATION}/processors/{PROCESSOR_ID}/humanReviewConfig"

    with open(DOCUMENT_PATH, "rb") as image:
        image_content = image.read()

    # Read the file into memory
    document = {"content": image_content, "mime_type": "image/tiff"}

    # Configure the process request
    request = {"human_review_config": name, "document": document}

    # Recognizes text entities in the PDF document
    result = client.review_document(request=request)
    
    print(f"Human Review Triggered for the document.\n{result.operation}\n")

    return result

In [ ]:
result = review_document_sample()

# Human in the loop (HITL) Operation

**Only complete this section if a HITL Operation is triggered.** </br>

In [ ]:
lro = "LONG_RUNNING_OPERATION" # LRO printed in the previous cell ex. projects/660199673046/locations/us/operations/174674963333130330

In [ ]:
client = documentai.DocumentProcessorServiceClient()
operation = client._transport.operations_client.get_operation(lro)
if operation.done:
    print("HITL location: {} ".format(str(operation.response.value)[5:-1]))
else:
    print('Waiting on human review.')

In [ ]:
!gsutil cp "HITL_LOCATION" response.json # Location printed above ex. gs://gcs_bucket/receipt-output/174674963333130330/data-00001-of-00001.json

In [ ]:
with open("response.json", "r") as file:
    import json
    entities = {}
    data = json.load(file)
    for entity in data['entities']:
        if 'mentionText' in entity:
            entities[entity['type']] = entity['mentionText']
        else:
            entities[entity['type']] = ""
    
    for t in entities:
        print("{} : {}\n ".format(t, entities[t]))